In [ ]:
load('https://raw.githubusercontent.com/cerickson30/qBound/main/spectator_floor_display_functions.py')

In [ ]:
print('Importing full uspcm_dict...')

full_uspcm_dict = get_full_uspcm_dict()
    
print('Full uspcm_dict has been imported.')

In [ ]:
def Glabel(G):
    return G.canonical_label().graph6_string()

def edgeclasses(G):
    Aut = G.automorphism_group()
    needs = {(xx[0], xx[1]): True for xx in G.edges()}
    while needs:
        anedge = next(iter(needs))
        yield anedge
        for xx in Aut.orbit(anedge, action='OnPairs'):
            if (xx[0], xx[1]) in needs:
                del needs[(xx[0], xx[1])]
            if (xx[1], xx[0]) in needs:
                del needs[(xx[1], xx[0])]

def deletions(G):
    for ed in edgeclasses(G):
        H = G.copy()
        H.delete_edge(ed)
        yield Glabel(H)

def contractions(G):
    # Also covers isolated vertex deletion
    for ed in edgeclasses(G):
        H = G.copy()
        H.contract_edge(ed)
        yield Glabel(H)
    if 0 in G.degree():
        H = G.copy()
        H.delete_vertex(G.degree().index(0))
        yield Glabel(H)

Algo to check for minor_minimality:

    for H in deletion(G) union contraction(G):
    
        if get_spectator_floor(H) == get_spectator_floor(G):
        
            G is not minor minimal
            
            break
            
    else:
    
        G is minor minimal

In [ ]:
def check_minimality(G, uspcm_dict):

    G, g6_str = get_canonical_graph(G)
    G_spec_floor = get_spectator_floor(G, uspcm_dict)

#     print(g6_str, G_spec_floor)

    for H in deletions(G):
#         print(f'Checking {H}')
        if get_spectator_floor(H, uspcm_dict) == G_spec_floor:
            # G is not minor minimal
            return None
    for H in contractions(G):
#         print(f'Checking {H}')
        if get_spectator_floor(H, uspcm_dict) == G_spec_floor:
            # G is not minor minimal
            return None
    else:
        # G is minor minimal
        return (g6_str, G_spec_floor)

In [ ]:
def progressBar(iterable, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    import datetime
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    total = len(iterable)
    # Progress Bar Printing Function
    def printProgressBar(iteration):
        try:
            percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
            filledLength = int(length * iteration // total)
        except ZeroDivisionError:
            percent = 100
            filledLength = length
        
        bar = fill * filledLength + '-' * (length - filledLength)
        print(f'\r{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} {prefix} |{bar}| {percent}% {suffix}', end = printEnd, flush=False)
    # Initial Call
    printProgressBar(0)
    # Update Progress Bar
    for i, item in enumerate(iterable):
        yield item
        printProgressBar(i + 1)
    # Print New Line on Complete
    print()
    

def determine_minimals(nn, minimals_dict=None, uspcm_dict=None, save=False, path_prefix='data'):
    
    if uspcm_dict is None:
        uspcm_dict = get_full_uspcm_dict()
        
    if minimals_dict is None:
        minimals_dict_keys = [f'{kk}_spectators' for kk in range(10)]
        minimals_dict = dict(zip(minimals_dict_keys, [set() for kk in range(10)]))
        
    
    for edge_key in uspcm_dict.get(f'{nn}_verts'):
        if save==True:
            one_percent = max(len(uspcm_dict.get(f'{nn}_verts').get(edge_key)) // 100, 1)
            fraction_percent = max(len(uspcm_dict.get(f'{nn}_verts').get(edge_key)) // 1000, 1)

            if fraction_percent < 500:
                save_percent = one_percent
            else:
                save_percent = fraction_percent
            
            
        num_edges = edge_key.split('_')[0]
        print(f'Working on graphs on {nn} vertices and {num_edges} edges...')
        
        num_graphs_worked = 0
        
        for g6_str in progressBar(uspcm_dict.get(f'{nn}_verts').get(edge_key), 
                            prefix = f"2nd pass: nn={nn}, ee={num_edges}:", 
                            suffix = '', length = 40):
            if Graph(g6_str).is_connected():
#                 print(g6_str)
                result = check_minimality(g6_str, uspcm_dict)
                if result is not None:
                    G_spec_num = result[1]
                    minimals_dict.get(f'{G_spec_num}_spectators').add(g6_str)

            num_graphs_worked += 1
                
            if num_graphs_worked % save_percent == 0:
                with open(path_prefix +
                  f'/minimals_dict/minimals_dict_{nn}_verts_{num_edges}_edges.txt', 'w') as outfile:
                    outfile.write(str(minimals_dict))
                    
    return minimals_dict

In [ ]:
path_prefix = 'data'

minimals_dict_keys = [f'{kk}_spectators' for kk in range(10)]
minimals_dict = dict(zip(minimals_dict_keys, [set() for kk in range(10)]))
minimals_dict['0_spectators'] = {'@'}

for nn in range(1, 11):
    minimals_dict = determine_minimals(nn, minimals_dict, full_uspcm_dict, save=True, path_prefix=path_prefix)
    
minimals_dict

with open(path_prefix + f'/minimals_dict.txt', 'w') as outfile:
    outfile.write(str(minimals_dict))